# Draft as of 06/26/2021

In [1]:
import pandas as pd

In [8]:
# Load raw datasets for inspection: 
raw_harvard_election_returns_main = pd.read_csv('./data/HarvardData_countypres_2000-2020.csv')
raw_nytimes_dataset = pd.read_csv('./data/NYTimes_covidcasedata.csv')
raw_facebook_social_connectedness_index = pd.read_csv('./data/Facebook_Cornell_ERS_SocialConnectednessIndex.csv')

In [9]:
raw_nytimes_dataset.head()

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0.0
1,2020-01-22,Snohomish,Washington,53061.0,1,0.0
2,2020-01-23,Snohomish,Washington,53061.0,1,0.0
3,2020-01-24,Cook,Illinois,17031.0,1,0.0
4,2020-01-24,Snohomish,Washington,53061.0,1,0.0


In [10]:
raw_harvard_election_returns_main.head()

,year,state,state_po,county_name,county_fips,office,candidate,party,candidatevotes,totalvotes,version,mode
0,2000,ALABAMA,AL,AUTAUGA,1001.0,PRESIDENT,AL GORE,DEMOCRAT,4942.0,17208.0,20191203,TOTAL
1,2000,ALABAMA,AL,AUTAUGA,1001.0,PRESIDENT,GEORGE W. BUSH,REPUBLICAN,11993.0,17208.0,20191203,TOTAL
2,2000,ALABAMA,AL,AUTAUGA,1001.0,PRESIDENT,RALPH NADER,GREEN,160.0,17208.0,20191203,TOTAL
3,2000,ALABAMA,AL,AUTAUGA,1001.0,PRESIDENT,OTHER,OTHER,113.0,17208.0,20191203,TOTAL
4,2000,ALABAMA,AL,BALDWIN,1003.0,PRESIDENT,AL GORE,DEMOCRAT,13997.0,56480.0,20191203,TOTAL


In [11]:
raw_facebook_social_connectedness_index.head()
# Social Index Density within 50 mile, 100 mile and 500 mile radius:

,FIPS,sh050m,sh100m,sh500m,income,migration,birth,death,education,unemployment,election
0,1001,0.206411,0.350840,0.291315,0.397313,0.407286,0.226810,-0.287958,0.653881,-0.056201,0.537613
1,1003,-0.099281,-0.641534,-0.209279,0.551486,1.643524,-0.174324,-0.358580,1.036397,0.052797,0.811953
2,1005,0.359257,0.516236,0.666761,-1.290785,-1.386023,-0.415005,-0.005468,-0.993064,1.687769,-0.866032
3,1007,1.582025,1.260516,0.917058,-0.494129,-0.058211,0.186697,-0.005468,-1.067442,0.652287,0.713348
4,1009,1.582025,1.260516,0.917058,-0.171091,-0.073473,0.226810,0.135776,-0.950562,0.107296,1.531833
